In [1]:
import os
import pandas as pd
import textwrap
import google.generativeai as genai
from dotenv import load_dotenv
from IPython.display import display
from IPython.display import Markdown

/Users/winnietu/Desktop/programming/google_devpost/devpost_google/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load the environment variables
load_dotenv()
# fetch an environment variable.
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

LLM 改寫 Query

In [7]:
# prompt for the User imput
def make_prompt_user(query):
    prompt = ("""
                You are excel at extracting main points from a paragraph and further generating keywords for googling. 
                Your job is to discern the users' intent from the paragraph and formulate keywords in both input language and English, 
                producing at least 5 distinct results seperatly.
                These keywords are for getting most relevant results from Google search.
                Optimizing these keywords according to the rules of each different language systems.
                If users' intent is not clear, you should ask for more information.
                If the user requests information unrelated to AI tool recommendations, return 
                "Sorry, I can only search for information about AI tools".
                For information related to tutorials, append Medium, Reddit, Youtube, Wiki or other reputable journalism sites to the 
                end of the search query. Ensure that the search results of queries are relevant to the paragraph and 
                include tool tutorials. Finally, return the finalized keywords in following type.
                QUESTION: '{query}'
                ANSWER: keyword1\nkeyword2\nkeyword3
                """).format(query=query)
    return prompt

In [22]:
# start with gemini model
model = genai.GenerativeModel('gemini-1.5-pro-latest')

# prompt = "我需要一個AI工具來幫助我安排行程，需要詳細的教學，有什麼推薦的工具嗎？最好是免費的工具"
prompt = "我要去韓國玩五天，需要一個可以幫我安排行程的AI工具，有什麼推薦的工具嗎？須附上使用步驟，最好是免費的"
# prompt = "I need an AI tool to help me plan my trip. I need detailed tutorials. Any recommended tools? Preferably free tools."

response = model.generate_content(
    make_prompt_user(prompt),
    generation_config={
        'temperature': 0.5,
        'top_k': 90,
    }
)

print(response.text)

QUESTION: '我要去韓國玩五天，需要一個可以幫我安排行程的AI工具，有什麼推薦的工具嗎？須附上使用步驟，最好是免費的'
ANSWER: 
**中文**
keyword1: 韩国 旅游 行程安排 AI 工具 免费 教程
keyword2: 五天 韩国 行程 定制 app 推荐 知乎
keyword3: 智能 旅行助手  韩国 自由行  规划  攻略  小红书
keyword4: 最佳  韩国 旅行路线  生成器  免费  豆瓣
keyword5:  AI 旅行规划 网站  韩国  中文  使用说明 

**English**
keyword1:  Korea trip itinerary AI planner free tutorial Reddit
keyword2:  5 days Korea itinerary app recommendation free Youtube
keyword3: best AI travel assistant Korea itinerary generator free
keyword4:  plan my trip to Korea AI tool step-by-step Medium
keyword5: free Korea travel itinerary planning website tutorial Wiki



In [ ]:
# response.text.split('\n')[1].split(': ')[1]

LLM 將爬蟲結果整理成 markdown 格式

In [9]:
# libraries for web scraping
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

Web crawler

In [23]:
# prompt for turn summary of website info into markdown file
def make_prompt_mdfile(query):
    prompt = ("""
                You are good at summarizing information of the website and return the results in markdown format.
                A markdown document should outline the main features of the answer for users. 
                The document should be structured as follows:
                Title: Use an H1 tag (#) for the title.
                Use H3 tags (###) for the sections and H4 tags (####) for sub-sections.
                Sections should be clearly labeled and easy to navigate.
                Section 1. Introduction
                Provide a brief overview of the user queries and information of the results.
                Section 2. Recommend List
                Each feature should be a bullet point, and any sub-features should be sub-bullets.
                Section 3. Installation Guide
                The steps for installation should be listed in a numbered list. 
                If there are any code snippets required for installation, please include them using code blocks.
                Section 4. Usage
                This section should explain how to use the software product. 
                Any steps should be in a numbered list, and include code snippets where necessary.
                Section 5. FAQ
                Each question should be a bullet point with an H5 tag (#####) for the question, and the answer should be written directly below the question without any special formatting.

                Please ensure that the document is easy to read and navigate, with clear headings and subheadings, bullet points for key points, and numbered lists for any step-by-step instructions."
                Website_html: '{query}'
                ANSWER:
                """).format(query=query)
    return prompt

In [30]:
# 從關鍵字搜尋網頁
def search_webpage(query):
    url = 'https://www.google.com/search?q=' + query
    return url

# 從給定的URL抓取網頁內容
def fetch_webpage(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text

# 從HTML中解析內容
def parse_html(html):
    soup = BeautifulSoup(html, 'html.parser')

    content = ' '.join([p.text for p in soup.find_all('p')])
    content = soup.title.string + ' ' + content
    return content

def web_to_md(url):
    html = fetch_webpage(url)
    content = parse_html(html)
    response = model.generate_content(
        make_prompt_mdfile(content),
        generation_config={
            'temperature': 0.5,
            'top_k': 50,
            'max_output_tokens': 1280
        }
    )
    print(response.text)

In [ ]:
options = Options()
options.binary_location = '../work/'
driver = webdriver.Chrome(options = options)

driver.get(search_webpage('AI tools for trip planning tutorial'))

# 等待页面加载
wait = WebDriverWait(driver, 10)

In [31]:
web_to_md('https://www.geeksforgeeks.org/ai-chatbots-for-travel-planning/')

# AI Chatbots for Travel Planning

## Introduction

This document summarizes the content of an article about AI chatbots designed for travel planning. The article highlights the benefits of using these chatbots to streamline the often tedious process of planning a trip. 

## Recommend List

Here is a list of AI-powered travel chatbots discussed in the article:

* **Layla (formerly Roam Around):**
    * User-friendly interface.
    * Creates personalized itineraries for popular destinations.
    * Requires minimal input: destination, travel dates, and preferences.
    * Link: [https://justasklayla.com/](https://justasklayla.com/)

* **Wonderplan:**
    * Budget-friendly option.
    * Provides detailed cost breakdowns for various activities.
    * Allows customization of itineraries by adding or removing items.
    * Link: [https://wonderplan.ai/](https://wonderplan.ai/)

* **Trip Planner AI:**
    * Offers unique features like itinerary suggestions based on TikTok and Instagram content.